<!--NAVIGATION-->

<a href="https://colab.research.google.com/github/bpesquet/machine-learning-katas/blob/master/classic-datasets/Iris.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Guided ML With The Iris Dataset

| Learning type | Activity type | Objective |
| - | - | - |
| Supervised | Multiclass classification | Identify a flower's class |

Contents:
1. Loading the data
2. Setting up supervised learning problem (selecting features)
3. Creating a first model
    - Creating train and test datasets
    - Normalizing train and test
    - Fitting and predicting
4. Evaluate the frist model predictions
5. Crossvalidation of the model
6. Creating an end to end ML pipeline
    - Train/Test Split
    - Normalize
    - Crossvalidations
    - Model
    - fitting and predicting

## Instructions with NBGrader removed

Complete the cells beginning with `# YOUR CODE HERE` and run the subsequent cells to check your code.

## About the dataset

[Iris](https://archive.ics.uci.edu/ml/datasets/iris) is a well-known multiclass dataset. It contains 3 classes of flowers with 50 examples each. There are a total of 4 features for each flower.

![](./classic-datasets/images/Iris-versicolor-21_1.jpg)

## Package setups

1. Run the following two cells to initalize the required libraries. 

In [1]:
#to debug package errors
import sys
sys.path
sys.executable

'/home/klarakhalo/anaconda3/bin/python'

In [2]:
# Import needed packages
# You may add or remove packages should you need them
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline

# Set random seed
np.random.seed(0)

# Display plots inline and change plot resolution to retina
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Set Seaborn aesthetic parameters to defaults
sns.set()

## Step 1: Loading the data

1. Load the iris dataset using ```datasets.load_iris()```
2. Investigate the data structure with ```.keys()```
3. Construct a dataframe from the dataset
4. Create a 'target' and a 'class' column that contains the target names and values
5. Display a random sample of the dataframe 

In [14]:
#Your code here.
iris = datasets.load_iris()
print(iris.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [39]:
iris["target_names"]

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [44]:
iris["feature_names"]

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [58]:
#your code here
iris_df = pd.read_csv('../data/iris.data', header=None)
iris_df["target"] = iris["target"]
iris_df.columns = ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Class", "Target"]
iris_df.sample(10)

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class,Target
114,5.8,2.8,5.1,2.4,Iris-virginica,2
62,6.0,2.2,4.0,1.0,Iris-versicolor,1
33,5.5,4.2,1.4,0.2,Iris-setosa,0
107,7.3,2.9,6.3,1.8,Iris-virginica,2
7,5.0,3.4,1.5,0.2,Iris-setosa,0
100,6.3,3.3,6.0,2.5,Iris-virginica,2
40,5.0,3.5,1.3,0.3,Iris-setosa,0
86,6.7,3.1,4.7,1.5,Iris-versicolor,1
76,6.8,2.8,4.8,1.4,Iris-versicolor,1
71,6.1,2.8,4.0,1.3,Iris-versicolor,1


### Question
Find the X and y values we're looking for. Notice that y is categorical and thus, we could **one-hot encode it** if we are looking at **class** or we can just pick **target**. In order to one hot encode we have  to re-shape `y` it using the **.get_dummies** function. 

#### For the purpose of this exercise, do not use hot encoding, go only for target but think about if you have to drop it somewhere or not...

In [164]:
# YOUR CODE HERE
X = iris_df.drop("Target", axis=1)
y = iris_df["Target"]
X.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [165]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64

In [166]:
type(y)

pandas.core.series.Series

In [167]:
type(X)

pandas.core.frame.DataFrame

## Step 2: Setting up supervised learning problem (selecting features)

Feature selection is an essential step in improving a model's perfromance. In the first version of the model we will use the 'sepal length' and 'sepal width' as predicting features. Later we will see the effect of adding additional features.

1. Assign the values of the 'target' to Y as a numpy array
2. Assign the remaining feature values to X as a numpy array
3. Check the shape of X and Y. Check the first few values.
    - Can we confirm our X and Y are created correctly?

In [168]:
#your code here
Y = np.array(y.values)
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [169]:
print(Y.shape)
Y[:5]

(150,)


array([0, 0, 0, 0, 0])

In [170]:
X = X.drop(["Petal Length", "Petal Width", "Class"], axis=1)

In [171]:
X = X.values

In [172]:
print(X.shape)
X[:5]

(150, 2)


array([[5.1, 3.5],
       [4.9, 3. ],
       [4.7, 3.2],
       [4.6, 3.1],
       [5. , 3.6]])

## Step 3: Creating the first model

In lecture we learned about creating a train and test datasets, normalizing, and fitting a model. In this step we will see how to build a simple version of this.

We have to be careful when constructing our train and test datasets. First, when we create train and test datasets we have to be careful that we always have the same datapoints in each set. Otherwise our results won't be reproduceable or we might introduce a bias into our model.

We also need to be attentive to when we normalize the data. What would be the effect of normalizing the data (i.e. with StandardScaler to a range between 0 - 1) before we create our train and test sets? Effectively we would use information in the test set to structure the values in the training set and vice versa. ***Therefore normalizing train and test independently is the preferred method.***

1. Create X_train, X_test, Y_train, Y_test using ```train_test_split()``` with an 80/20 train/test split. Look in the SKLearn documentation to understand how the function works.
    - Inspect the first few rows of X_train.
    - Run the cell a few times. Do the first few rows change?
    - What option can we use in ```train_test_split()``` to stop this from happening?
2. Normalize the train and test datasets with ```StandardScaler```
    - We can fit the transform with ```.fit()``` and ```.transform()``` to apply it. Look in the documentation for an esample of how to do this.
    - Does it make sense to normalize Y_train and Y_test?
3. Initalize a ```LogisticRegression()``` model and use the ```.fit()``` method to initalize the first model.
    - We will pass the X_train and Y_train variables to the ```.fit()``` method.
    - Once the model is fit, use the ```.predict()``` with the X_test and save the output as predictions.

In [295]:
#split train and test data 80/20
#your code here
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=42)
X_train[:10]

array([[4.6, 3.6],
       [5.7, 4.4],
       [6.7, 3.1],
       [4.8, 3.4],
       [4.4, 3.2],
       [6.3, 2.5],
       [6.4, 3.2],
       [5.2, 3.5],
       [5. , 3.6],
       [5.2, 4.1]])

In [296]:
from sklearn.preprocessing import StandardScaler

In [297]:
scaler = StandardScaler()
X_train = scaler.fit(X_train).transform(X_train)
print(X_train[:10])

[[-1.47393679  1.22037928]
 [-0.13307079  3.02001693]
 [ 1.08589829  0.09560575]
 [-1.23014297  0.77046987]
 [-1.7177306   0.32056046]
 [ 0.59831066 -1.25412249]
 [ 0.72020757  0.32056046]
 [-0.74255534  0.99542457]
 [-0.98634915  1.22037928]
 [-0.74255534  2.34515281]]


In [298]:
scaler = StandardScaler()
X_test = scaler.fit(X_test).transform(X_test)
print(X_test[:10])

[[ 0.14443512 -0.63451517]
 [-0.33701527  2.00929805]
 [ 2.07023667 -1.16327782]
 [ 0.02407252 -0.37013385]
 [ 0.98697329 -0.63451517]
 [-0.69810306  0.95177276]
 [-0.45737787 -0.37013385]
 [ 1.10733589  0.15862879]
 [ 0.26479771 -2.22080311]
 [-0.21665267 -0.8988965 ]]


In [299]:
#initalize and fit with Logistic Regression
#your code here
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

LogisticRegression()

In [300]:
predictions = logisticRegr.predict(X_test)
predictions

array([1, 0, 2, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0, 1,
       0, 1, 2, 2, 2, 2, 0, 0])

## Step 4: Evaluate the frist model's predictions

We will learn more about how to evaluate the performance of a classifier in later lessons. For now we will use % accuracy as our metric. It is important to know that this metric only helps us understand the specific performance of our model and not, for example, where we can improve it, or where it already perfoms well.

1. Use ```.score()``` to evaluate the performance of our first model.

In [301]:
#evaluating the performace of our first model
#your code here
print(logisticRegr.score(X_train, y_train))
print(logisticRegr.score(X_test, y_test))

0.7916666666666666
0.8333333333333334


## Step 5: Question your results. 
What accuracy did you achieve? Is it 70, 90%? Anything above 70% is a good fit for our first result. How do we know it is reproducible? **If we run the model again and our performance is 85%, which one is correct**? And what about improving our model? 

## However ...
There is one crucial mistake that has been made in the exercise above -even if we achieved great results-. Can you spot it? You can go back to the lecture slides for inspiration. 

*Type your answer here...*
***
***No idea***

## Optional:
Repeat the cells you need to change in the exercise and run the classifier again. What is the new accuracy and why is this better?

In [1]:
#your code here